In [ ]:
# v8+ should be fine
!java -version

java version "19.0.1" 2022-10-18
Java(TM) SE Runtime Environment (build 19.0.1+10-21)
Java HotSpot(TM) 64-Bit Server VM (build 19.0.1+10-21, mixed mode, sharing)


In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
import pandas as pd

splitter_jar_path = './tools/sentence-splitter/SentenceSplitter.jar'
splitter_model_path = './tools/sentence-splitter/es-sentence-splitter-model-spaccc.bin'
text_path = './data/symptemist_train/subtask1-ner/txt/'
annotations_path = './data/symptemist_train/subtask1-ner/brat/'
output_path = './data/processed/'
sents_per_batch = 1 # max number of sentences in each output file

In [ ]:
for file_name in os.listdir(text_path):
    file_name_no_ext = file_name.rstrip('.txt')
    p = Popen(['java', '-jar', splitter_jar_path , f'{text_path}/{file_name}', splitter_model_path], stdout=PIPE, stderr=STDOUT, text=True)

    print("Processing: ", file_name_no_ext)

    annotations = pd.read_csv(f'{annotations_path}/{file_name_no_ext}.ann', sep='\t', names=['ann_type', 'entity_type', 'text'], encoding='utf-8')

    # drop annotator comments and attribute annotations
    annotations.drop(annotations[annotations['ann_type'].str.startswith('#') | annotations['ann_type'].str.startswith("A")].index, inplace=True)

    print(annotations)

    annotations['start_pos'] = annotations['entity_type'].transform(lambda v: int(v.split()[1]))
    annotations['end_pos'] = annotations['entity_type'].transform(lambda v: int(v.split()[2]))
    annotations['entity_type'] = annotations['entity_type'].transform(lambda v: v.split()[0])
    annotations.sort_values(by=['start_pos'], inplace=True, ignore_index=True)

    total_chars = 0
    annotation_index = 0
    total_annotations = len(annotations)
    line_number = 0
    batch = []
    batch_number = 1
    for line in p.stdout:
        line_number += 1
        batch.append(line)

        if line_number % sents_per_batch == 0:
            batch_number += 1
            current_batch_annotation_indecies = []
            text = ''.join(batch)
            max_index = total_chars + len(text) - 1;
            while annotation_index < total_annotations and annotations.iloc[annotation_index]['start_pos'] <= max_index:
                annotation_text = annotations.iloc[annotation_index]['text']

                new_start_pos = text.find(annotation_text)
                annotations.at[annotation_index, 'start_pos'] = new_start_pos
                annotations.at[annotation_index, 'end_pos'] = new_start_pos + len(annotation_text)
                current_batch_annotation_indecies.append(annotation_index)
                annotation_index += 1

            with open(f'{output_path}/splits_{sents_per_batch}/{file_name_no_ext}-b-{batch_number}.txt', 'w', encoding='utf-8') as out_file:
                out_file.write(text)

            current_batch_annotations = annotations.iloc[current_batch_annotation_indecies,].copy().reset_index(drop=True)
            current_batch_annotations['entity_type_with_positions'] = None
            if not current_batch_annotations.empty:
                current_batch_annotations['row_name'] = current_batch_annotations.apply(lambda row: row.name + 1, axis=1)
                current_batch_annotations['ann_type'] = current_batch_annotations.apply(lambda row: f'{row["ann_type"].rstrip("0123456789")}{row.name + 1}', axis=1)
                current_batch_annotations['entity_type_with_positions'] = current_batch_annotations.apply(lambda row: f'{row["entity_type"]} {row["start_pos"]} {row["end_pos"]}', axis=1)
                current_batch_annotations.drop(columns=['start_pos', 'end_pos', 'entity_type'], inplace=True)

            current_batch_annotations.to_csv(f'{output_path}/splits_{sents_per_batch}/{file_name_no_ext}-b-{batch_number}.ann', sep='\t', header=False, index=False, columns=['ann_type', 'entity_type_with_positions', 'text'], encoding='utf-8')

            total_chars += len(text) -1
            batch.clear()

Processing:  es-S0004-06142007000600016-2
  ann_type      entity_type                                               text
0       T1    SINTOMA 30 66               tumoración en polo superior de teste
1       T2  SINTOMA 558 594               tumoración nodular en cola epidídimo
2       T3  SINTOMA 225 284  masa de 1cm aproximado de diámetro dependiente...
3       T4  SINTOMA 412 424                                       masa nodular
4       T5  SINTOMA 760 777                                  formación nodular
Processing:  es-S0004-06142007000700013-1
   ann_type        entity_type  \
0        T1    SINTOMA 399 421   
1        T2    SINTOMA 566 600   
2        T3    SINTOMA 615 631   
3        T4    SINTOMA 634 654   
4        T5    SINTOMA 659 683   
5        T6    SINTOMA 746 796   
6        T7    SINTOMA 863 876   
7        T8  SINTOMA 2582 2650   
8        T9    SINTOMA 520 542   
9       T10    SINTOMA 544 561   
10      T11  SINTOMA 1202 1229   
11      T12  SINTOMA 1419 1489   
